# Laboratorio 9

### Integrantes:

 - BRANDON JAVIER REYES MORALES
 - CARLOS ALBERTO VALLADARES GUERRA 
 - JUAN PABLO SOLIS ALBIZUREZ

### **Task 1: Teoria**

1. Diga cuál es la diferencia entre Modelos de Markov y Hidden Markov Models
- En los modelos de markov se representan sistemas donde el estado actual depende únicamente del estado anterior. Estos modelos asumen que todos los estados son observables. Por otro lado, los Hidden Markov Models suponen que los estados verdaderos del sistema son ocultos y no se pueden observar directamente. En su lugar, se observan señales o emisiones que dependen probabilísticamente de estos estados ocultos.

2. Investigue qué son los factorial HMM (Hidden Markov Models)
- Los factorial Hidden Markov Models  son una generalización de los Hidden Markov Models tradicionales. En lugar de tener una sola cadena de estados ocultos, los FHMM utilizan múltiples cadenas ocultas que evolucionan de forma paralela e independiente. Cada cadena representa una parte diferente del sistema oculto, y las observaciones se generan combinando la información de todas las cadenas. Esto permite modelar fenómenos más complejos, ya que se puede capturar la influencia simultánea de múltiples factores ocultos.

3. Especifique en sus propias palabras el algoritmo Forward-Backward para HMM
- El algoritmo Forward-Backward es una técnica utilizada en los HMM para calcular la probabilidad de que el sistema esté en un cierto estado en un momento dado, considerando toda la secuencia de observaciones. Este algoritmo tiene dos fases: la fase "Forward" calcula, hacia adelante en el tiempo, la probabilidad acumulada de haber alcanzado cada estado, dada la secuencia de observaciones hasta ese punto. La fase "Backward" hace el cálculo inverso: determina la probabilidad de observar la secuencia restante a partir de un estado específico, hacia atrás en el tiempo. 

4. En el algoritmo de Forward-Backward, ¿por qué es necesario el paso de Backward?
- El paso de Backward es necesario porque nos permite tener en cuenta no solo el pasado, sino también el futuro en nuestras estimaciones. Si solo usamos el paso Forward, solo consideraríamos la información hasta el momento actual, lo cual puede ser insuficiente para tomar decisiones precisas. Al incluir el paso Backward, integramos también las observaciones futuras, lo cual mejora significativamente la precisión de la estimación de los estados ocultos. 

### Task 2

In [2]:
import numpy as np
import random

class HMM:
    def __init__(self, states, observations, initial_prob, transition_prob, emission_prob):
        self.states = states
        self.observations = observations
        self.initial_prob = initial_prob
        self.transition_prob = transition_prob
        self.emission_prob = emission_prob

    def generate_sequence(self, length):
        sequence = []
        current_state = random.choices(self.states, weights=[self.initial_prob[s] for s in self.states])[0]
        
        for _ in range(length):
            obs = random.choices(self.observations, weights=[self.emission_prob[current_state][o] for o in self.observations])[0]
            sequence.append(obs)
            current_state = random.choices(self.states, weights=[self.transition_prob[current_state][s] for s in self.states])[0]
        
        return sequence

    def forward(self, obs_sequence):
        forward_probs = [{}]
        for state in self.states:
            forward_probs[0][state] = self.initial_prob[state] * self.emission_prob[state][obs_sequence[0]]

        for t in range(1, len(obs_sequence)):
            forward_probs.append({})
            for curr_state in self.states:
                prob_sum = sum(
                    forward_probs[t-1][prev_state] * self.transition_prob[prev_state][curr_state]
                    for prev_state in self.states
                )
                forward_probs[t][curr_state] = prob_sum * self.emission_prob[curr_state][obs_sequence[t]]
        
        return forward_probs

    def backward(self, obs_sequence):
        backward_probs = [{} for _ in range(len(obs_sequence))]
        for state in self.states:
            backward_probs[-1][state] = 1

        for t in reversed(range(len(obs_sequence) - 1)):
            for curr_state in self.states:
                backward_probs[t][curr_state] = sum(
                    self.transition_prob[curr_state][next_state] *
                    self.emission_prob[next_state][obs_sequence[t+1]] *
                    backward_probs[t+1][next_state]
                    for next_state in self.states
                )
        
        return backward_probs

    def compute_state_probabilities(self, obs_sequence):
        forward_probs = self.forward(obs_sequence)
        backward_probs = self.backward(obs_sequence)

        state_probs = [{} for _ in range(len(obs_sequence))]

        for t in range(len(obs_sequence)):
            total_prob = sum(
                forward_probs[t][state] * backward_probs[t][state]
                for state in self.states
            )
            for state in self.states:
                state_probs[t][state] = (forward_probs[t][state] * backward_probs[t][state]) / total_prob
        
        return state_probs


# Parámetros del modelo
states = ['Sunny', 'Rainy']
observations = ['Sunny', 'Rainy']

initial_prob = {'Sunny': 0.5, 'Rainy': 0.5}

transition_prob = {
    'Sunny': {'Sunny': 0.8, 'Rainy': 0.2},
    'Rainy': {'Sunny': 0.4, 'Rainy': 0.6}
}

emission_prob = {
    'Sunny': {'Sunny': 0.8, 'Rainy': 0.2},
    'Rainy': {'Sunny': 0.3, 'Rainy': 0.7}
}

# Crear HMM
hmm = HMM(states, observations, initial_prob, transition_prob, emission_prob)

# Generar secuencia
obs_sequence = hmm.generate_sequence(5)
print("Secuencia Generada:", obs_sequence)

# Forward
forward_probs = hmm.forward(obs_sequence)
print("\nProbabilidades Forward:")
for t, probs in enumerate(forward_probs):
    print(f"Tiempo {t}: {probs}")

# Backward
backward_probs = hmm.backward(obs_sequence)
print("\nProbabilidades Backward:")
for t, probs in enumerate(backward_probs):
    print(f"Tiempo {t}: {probs}")

# Cálculo de Probabilidades de Estado
state_probs = hmm.compute_state_probabilities(obs_sequence)
print("\nProbabilidades de Estado (Posteriores):")
for t, probs in enumerate(state_probs):
    print(f"Tiempo {t}: {probs}")


Secuencia Generada: ['Rainy', 'Rainy', 'Rainy', 'Rainy', 'Sunny']

Probabilidades Forward:
Tiempo 0: {'Sunny': 0.1, 'Rainy': 0.35}
Tiempo 1: {'Sunny': 0.044000000000000004, 'Rainy': 0.16099999999999998}
Tiempo 2: {'Sunny': 0.01992, 'Rainy': 0.07377999999999998}
Tiempo 3: {'Sunny': 0.009089600000000001, 'Rainy': 0.03377639999999999}
Tiempo 4: {'Sunny': 0.016625792, 'Rainy': 0.006625127999999998}

Probabilidades Backward:
Tiempo 0: {'Sunny': 0.028296800000000004, 'Rainy': 0.0583464}
Tiempo 1: {'Sunny': 0.06636, 'Rainy': 0.12628}
Tiempo 2: {'Sunny': 0.18200000000000005, 'Rainy': 0.266}
Tiempo 3: {'Sunny': 0.7000000000000002, 'Rainy': 0.5}
Tiempo 4: {'Sunny': 1, 'Rainy': 1}

Probabilidades de Estado (Posteriores):
Tiempo 0: {'Sunny': 0.1217018509375113, 'Rainy': 0.8782981490624887}
Tiempo 1: {'Sunny': 0.12557954695986226, 'Rainy': 0.8744204530401377}
Tiempo 2: {'Sunny': 0.15592673322173925, 'Rainy': 0.8440732667782608}
Tiempo 3: {'Sunny': 0.27365454786305243, 'Rainy': 0.7263454521369476}
T